# Schema extraction

## Schema definition



In [9]:
node_schema = {
    'name': '',
    'type': '',
    'values': ['']
}

# conditions and condition_values should be matched
# value should be in the values of the variable
edge_schema = {
    'conditions': [''],
    'condition_values': [''],
    'variable': '',
    'value': '',
    'probability': ''
}

In [10]:
import os

from zhipuai import ZhipuAI
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate

os.environ["ZHIPUAI_API_KEY"] = '8adac289dfd59cb2006ccea31a82efab.GhrrpvmaygKVqjxS'


llm = ChatZhipuAI(
    temperature=0,
    model="glm-4"
)

In [14]:
from model import Kimi

llm=Kimi()

In [15]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from idextraction import Node, List_of_Nodes

from typing import List


extract_template = """\
For the following text, identify the fundamental goal inplied by the context (which may not directly appear in the text), and extract every variable, action or event. If none is found, output a default node, with name and type filled with empty strings and values an empty list. Output using the following format:

name: Extract the core concept or variable\
related to decision-making, and output its name.
    
type: What is the nature of the variable? \
If it is an aleatory variable that cannot be intervened, output 'chance';\
if it can be intervened and represents a decision to be made, output 'decision';\
if it reflects the fundamental goal of decision-maker in the setting, output 'utility'.

values: Extract the possible values of the variable, complete with common knowledge if necessary, and output them as a Python list.

text: {text}

{format_instructions}
"""

# class Node(BaseModel):
#     variable_name: str = Field(description="Extract the core concept or variable related to decision-making, and output its name.")
#     variable_type: str = Field(description="What is the nature of the variable? If it is an aleatory variable that cannot be intervened, output 'chance'; if it can be intervened and represents a decision to be made, output 'decision'; if it reflects the fundamental goal of decision-maker in the setting, output 'utility'.")
#     values: list = Field(description="Extract the possible values of the variable, complete with common knowledge if necessary, and output them as a Python list. ")

# class List_of_Nodes(BaseModel):
#     node_list: List[Node]

parser = JsonOutputParser(pydantic_object=List_of_Nodes)

prompt = PromptTemplate(
    template=extract_template,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser


In [17]:
# test example
text = """Openings connecting upper and
lower floors within buildings compromise the in-
tegrity of fire compartments, potentially leading to
the spread of fire across multiple areas and floors.
Therefore, reliable fire separation measures should
be implemented in these connected spaces to pre-
vent the rapid upward spread of fire."""

contents = chain.invoke({"text": text})
contents['node_list']

[{'variable_name': 'Fire separation measures',
  'variable_type': 'decision',
  'values': ['implemented', 'not implemented']},
 {'variable_name': 'Spread of fire',
  'variable_type': 'chance',
  'values': ['rapid upward spread', 'no spread']},
 {'variable_name': 'Integrity of fire compartments',
  'variable_type': 'utility',
  'values': ['compromised', 'maintained']}]

In [3]:
def extract_node(text):
    parser = JsonOutputParser(pydantic_object=List_of_Nodes)

    prompt = PromptTemplate(
        template=extract_template,
        input_variables=["text"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | llm | parser
    contents = chain.invoke({"text": text})
    return contents['node_list']

1. read file, split into sentences/paragraphs
2. node extraction: from each sentence/paragraph, extract node info and save
3. node post-processing: combine synonymous nodes
4. arc extraction: for each node, search related texts and extract its **parent nodes**
5. probability judgement: for each value of ([parents], child) pair, assign conditional probability (verbal)
6. normalization: for each ([parents], child) pair and for each value of parents, normalize conditional probability (numerical)

In [4]:
from pprint import pprint

with open("E:/PhD_2/NLP/project/corpus/structure_dataset.txt", 'r') as f:
    texts = f.read().split()

for text in texts:
    pprint(extract_node(text))
    

[{'values': ['Calculated value based on the formula provided in the Canadian '
             'Wood Design Code'],
  'variable_name': 'Horizontal Displacement',
  'variable_type': 'utility'},
 {'values': ['Value calculated from the first term of the formula'],
  'variable_name': 'Displacement due to Shear and Bending Moment',
  'variable_type': 'chance'},
 {'values': ['Value calculated from the second term of the formula'],
  'variable_name': 'Displacement due to Anchor Pullout and Boundary '
                   'Compression',
  'variable_type': 'chance'},
 {'values': ['Value calculated from the third term of the formula'],
  'variable_name': 'Displacement due to Shear Deformation of Wood-based Panels '
                   'and Nail Deformation',
  'variable_type': 'chance'},
 {'values': ['Value calculated from the fourth term of the formula'],
  'variable_name': 'Displacement due to Shear Wall Base Rotation',
  'variable_type': 'chance'},
 {'values': ["Values given by the manufacturer's t

HTTPStatusError: Client error '429 Too Many Requests' for url 'https://open.bigmodel.cn/api/paas/v4/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429